<a href="https://colab.research.google.com/github/misbahsy/tafsir_semantic_search/blob/main/embedding_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain cohere pinecone-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 KB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 3.4 MB/s eta 0:00:00
  Created wheel for cohere: filename=cohere-3.2.6-cp38-cp38-linux_x86_64.whl size=14392 sha256=abdf46b6e5a2bd24535d1fb22eb7e6ed174ac2f3ccf4b6dc564c7882503d5184
  Stored in directory: /root/.cache/pip/wheels/3f/f2/7b/6a2486838e5176c43fcd17eb6c07d4705d3d15e539394f7cae
Successfully built cohere
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
api_key = 'cf643833-3fee-4700-9ac7-f0f90635a544'
cohere_api_key = 'use-your-api-key'

In [3]:
import cohere
co = cohere.Client(cohere_api_key)

In [4]:
import pinecone

index_name = 'tafsir'
pinecone.init(api_key, environment='us-west1-gcp')

In [5]:
# connect to index
index = pinecone.Index(index_name)

In [16]:
limit = 1600

def retrieve(query):
    xq = co.embed(
        texts=[query],
        model='multilingual-22-12',
        truncate='NONE'
    ).embeddings
    # search pinecone index for context passage with the answer
    xc = index.query(xq, top_k=3, include_metadata=True)
    contexts = [
        x['metadata']['text'] for x in xc['matches']
    ]

    # build our prompt with the retrieved contexts included
    prompt_start = (
        "Answer the Query based on the contexts, if it's not in the contexts say 'I don't know the answer'. \n\n"+
        "Context:\n"
    )
    prompt_end = (
        f"\n\nQuery: {query}\nAnswer in the language of Query, if Query is in English Answer in English."
    )
    # append contexts until hitting limit
    for i in range(1, len(contexts)):
        if len("\n\n---\n\n".join(contexts[:i])) >= limit:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts[:i-1]) +
                prompt_end
            )
            break
        elif i == len(contexts)-1:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts) +
                prompt_end
            )
    return prompt

In [17]:
query = "What does quran say about believers?"
query_with_contexts = retrieve(query)
query_with_contexts

"Answer the Query based on the contexts, if it's not in the contexts say 'I don't know the answer'. \n\nContext:\nقال تعالى -موبخا لهم على إقامتهم على ما هم عليه من الكفر والنفاق-: { أَوَلا يَرَوْنَ أَنَّهُمْ يُفْتَنُونَ فِي كُلِّ عَامٍ مَرَّةً أَوْ مَرَّتَيْنِ } بما يصيبهم من البلايا والأمراض، وبما يبتلون من الأوامر الإلهية التي يراد بها اختبارهم.\n{ ثُمَّ لا يَتُوبُونَ } عما هم عليه من الشر { وَلا هُمْ يَذَّكَّرُونَ } ما ينفعهم، فيفعلونه، وما يضرهم، فيتركونه.\nفالله تعالى يبتليهم -كما هي سنته في سائر الأمم- بالسراء والضراء وبالأوامر والنواهي ليرجعوا إليه، ثم لا يتوبون ولا هم يذكرون.\nوفي هذه الآيات دليل على أن الإيمان يزيد وينقص، وأنه ينبغي للمؤمن، أن يتفقد إيمانه ويتعاهده، فيجدده وينميه، ليكون دائما في صعود.\n(1/356)\n\n---\n\nثم ذكر عباده المؤمنين بما أنزل عليهم من كتابه، الذي أنزله على رسوله محمد صلى الله عليه وسلم، ليخرج الخلق من ظلمات الكفر والجهل والمعصية، إلى نور العلم والإيمان والطاعة، فمن الناس، من آمن به، ومنهم من لم يؤمن [به]، { وَمَنْ يُؤْمِنْ بِاللَّهِ وَيَعْمَلْ صَالِحً

In [18]:
!pip install -qU openai 

In [19]:
import openai

# get API key from top-right dropdown on OpenAI website
openai.api_key = "use-your-api-key"

In [22]:
def complete(prompt):
    # query text-davinci-003
    res = openai.Completion.create(
        engine='text-davinci-003',
        prompt=prompt,
        temperature=0,
        max_tokens=1000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    return res['choices'][0]['text'].strip()

In [23]:
complete(query_with_contexts)

'القرآن يقول أن المؤمنين يؤمنون بالكتاب الذي أنزل عليهم حقَّ تِلاوَتِهِ، ويشكرون نعمة الله التي أنعم عليهم، ويتقون يوما لا يقبل من الناس عدل ولا تنفعهم شفاعة.'